In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup, Comment
from tqdm import tqdm

In [2]:
SCORE_DIR = "E:/Projects/NBA Prediction/html"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [5]:
box_scores

['E:/Projects/NBA Prediction/html\\201410280LAL.html',
 'E:/Projects/NBA Prediction/html\\201410280NOP.html',
 'E:/Projects/NBA Prediction/html\\201410280SAS.html',
 'E:/Projects/NBA Prediction/html\\201410290BOS.html',
 'E:/Projects/NBA Prediction/html\\201410290CHO.html',
 'E:/Projects/NBA Prediction/html\\201410290DEN.html',
 'E:/Projects/NBA Prediction/html\\201410290IND.html',
 'E:/Projects/NBA Prediction/html\\201410290MEM.html',
 'E:/Projects/NBA Prediction/html\\201410290MIA.html',
 'E:/Projects/NBA Prediction/html\\201410290NYK.html',
 'E:/Projects/NBA Prediction/html\\201410290PHO.html',
 'E:/Projects/NBA Prediction/html\\201410290POR.html',
 'E:/Projects/NBA Prediction/html\\201410290SAC.html',
 'E:/Projects/NBA Prediction/html\\201410290TOR.html',
 'E:/Projects/NBA Prediction/html\\201410290UTA.html',
 'E:/Projects/NBA Prediction/html\\201410300CLE.html',
 'E:/Projects/NBA Prediction/html\\201410300DAL.html',
 'E:/Projects/NBA Prediction/html\\201410300LAC.html',
 'E:/Proje

In [6]:
def parse_html(box_score):
    with open(box_score, "r", encoding='utf-8') as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))

    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thread")]

    return soup, comments


In [7]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [8]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [9]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [14]:
base_cols = None
games = []

for box_score in tqdm(box_scores):
    box_score = box_scores[0]
    soup, comment = parse_html(box_score)
    line_score = read_line_score(comment)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]

    games.append(full_game)


In [11]:
games_df = pd.concat(games, ignore_index=True)

In [12]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,31.0,73.0,0.425,12.0,29.0,0.414,34.0,50.0,...,34.4,33.2,143.0,129.0,LAL,90,1,2015,2014-10-28,True
1,240.0,240.0,28.0,79.0,0.354,3.0,10.0,0.300,31.0,39.0,...,43.1,34.7,161.0,109.0,HOU,108,0,2015,2014-10-28,False


In [14]:
games_df.to_csv("nba_games.csv")